<a href="https://colab.research.google.com/github/jiin124/review_project/blob/main/modeling/%EC%95%84%EA%B3%A0%EB%8B%A4_%ED%98%B8%ED%85%94%EB%A6%AC%EB%B7%B0_Koelectra%2CRoBERTa_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
pip install datasets

In [ ]:
#한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager,rc

font_path="/content/drive/MyDrive/NanumSquareB.ttf"
font=font_manager.FontProperties(fname=font_path)

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font',family='NanumBarunGothic')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from typing import Dict, List, Set, Callable, Optional
plt.style.use('ggplot')

In [ ]:
import random
import os

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED']=str(seed)
  np.random.seed(seed)

seed_everything(42)

In [ ]:
#1
df=pd.read_csv('/content/hotel_review_clean.csv')
df.head()


In [ ]:
df['review'][0]

'아버님 서울 병원 방문 목적으로 혼자 숙박하셔야 되어 걱정 많았으나 사전 전화 문의에도 친절하셨고 사용하신 아버님도 매우 만족하셨습니다.'

# 데이터 나누기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,test=train_test_split(df,test_size=0.1,random_state=42)

print(train.shape)
print(test.shape)

(2715, 15)
(302, 15)


In [ ]:
test_rate=test['rate']
test=test.drop('rate',axis=1)


In [ ]:
print(test.shape)
print(test_rate.shape)

(302, 14)
(302,)


In [ ]:
display(train.head())
print()
display(test.head())
display(test_rate)

,star,Hotel_name,review_rate,review_grade,reviewer_name,trip_type,room_type,trip_date_year,trip_date_month,title,review,trip_day,review_date,datetime,rate
433,5,롯데호텔서울 이그제큐티브 타워,10.0,최고,Kiwook,커플/2인 여행객,이그제큐티브 그랜드 디럭스 트윈룸 (르 살롱 이용 포함),2022.0,7.0,최고의 날이었습니다.”,결혼식을 하고 1박을 롯데호텔에서 하게 되었는데 제 인생에서 최고의 하루였습니다. ...,1,2022-07-19,2022-07-19,5
1178,2,스테이 인터뷰 제주,3.6,NaN,So,나홀로 여행객,"부분 오션뷰 룸 (퀸베드, 금연)",2019.0,5.0,무난함”,방음이 거의 안된다고 보시면 됩니다. 리뷰를 보고 감안하고 갔으나 예약 시 조용한 ...,2,2019-05-05,2019-05-05,1
30,1,유원호텔_서초,5.6,NaN,Sang,출장 여행객,럭셔리 스위트,2021.0,11.0,서비스는 좋았으나 객실 노후 상태가 심함”,환불 조치하여 주었음,1,2021-11-29,2021-11-29,2
1239,3,블루스프링 부띠크 호텔,6.8,양호,GIL,커플/2인 여행객,스튜디오 더블,2021.0,10.0,위치는 좋지만 시설 위생상태x”,제목 그대로 위치만 좋은 곳,1,2022-09-06,2022-09-06,3
1422,5,랜딩관 제주신화월드 호텔 앤 리조트,10.0,최고,YOUNGMI,청소년 동반 가족 여행객,디럭스 패밀리 트윈,2020.0,9.0,9박10일 행복했던 가족여행 ”,"결혼 30주년 기념으로 제주도 9박 10일 가족여행, 숙소도 쾌적하고 편안한 행복한...",9,2020-10-07,2020-10-07,5


,star,Hotel_name,review_rate,review_grade,reviewer_name,trip_type,room_type,trip_date_year,trip_date_month,title,review,trip_day,review_date,datetime
1338,4,베니키아 호텔 제주,7.6,매우 좋음,동현,출장 여행객,패밀리 트윈,2022.0,8.0,출장”,출장용으로 싸고 무난,2,2022-09-20,2022-09-20
1230,3,블루스프링 부띠크 호텔,5.2,NaN,Hyesu,그룹 여행객,슈페리어 트윈,2022.0,9.0,위치만 좋은 모텔급 호텔”,추천 안 합니다,1,2022-09-30,2022-09-30
478,5,롯데호텔서울 이그제큐티브 타워,10.0,최고,HANA,커플/2인 여행객,이그제큐티브 그랜드 디럭스 트윈룸 (르 살롱 이용 포함),2021.0,4.0,최고의 프리미엄 명품서비스”,서비스의 디테일이 살아있다고 느꼈어요 비싼데 전체적으로 만족도는 매우 높아요... ...,1,2021-04-27,2021-04-27
44,1,호텔 컬리넌 건대 2호점,10.0,최고,현진,출장 여행객,프리미엄룸,2019.0,10.0,키가 1개인 것만 빼면 다좋음”,깨끗하고 조용하고 시설도 낡은 거 없이 깔끔함. 근데 키가 1개라 함께 다니는 사람...,3,2019-11-04,2019-11-04
1071,1,숨 게스트 하우스 제주공항,10.0,최고,HyunHo,나홀로 여행객,6 베드 도미토리 내 1인 예약 (남성 전용),2020.0,11.0,좋습니다”,좋습니다ㅏ.,2,2020-11-22,2020-11-22


1338    3
1230    2
478     5
44      5
1071    5
       ..
162     5
1591    5
2939    2
2913    1
871     1
Name: rate, Length: 302, dtype: int64

In [ ]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)
test_rate.to_csv('test_rate.csv',index=False)

In [ ]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
test_rate=pd.read_csv('/content/test_rate.csv')

# Modeling

In [ ]:
pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback,TrainingArguments,Trainer
from transformers.tokenization_utils_base import BatchEncoding
from transformers.trainer_utils import EvalPrediction
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from sklearn.metrics import accuracy_score

In [ ]:
MAX_LEN=128
VALID_SPLIT=0.1
EPOCHS=8
LEARNING_RATE=1e-5
DR_RATE=0.3
WARMUP_STEPS=500
WEIGHT_DECAY=0.01
METRIC='accuracy'

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#KoELECTRA Training
MODEL_NAME='koelectra'
MODEL_PATH='jaehyeong/koelectra-base-v3-generalized-sentiment-analysis'
BATCH_SIZE=64

# RoBERTa Training
# MODEL_NAME = 'roberta'
# MODEL_PATH = 'klue/roberta-large'
# BATCH_SIZE = 32

In [ ]:
# RoBERTa Training
MODEL_NAME = 'roberta'
MODEL_PATH = 'klue/roberta-large'
BATCH_SIZE = 32

# Load Datasets

In [ ]:
LABELS=[1,2,3,4,5]
id2label={idx:label for idx,label in enumerate(LABELS)}
label2id={label:idx for idx,label in enumerate(LABELS)}
print(train.shape)
train.head()

(2715, 15)


,star,Hotel_name,review_rate,review_grade,reviewer_name,trip_type,room_type,trip_date_year,trip_date_month,title,review,trip_day,review_date,datetime,rate
0,5,롯데호텔서울 이그제큐티브 타워,10.0,최고,Kiwook,커플/2인 여행객,이그제큐티브 그랜드 디럭스 트윈룸 (르 살롱 이용 포함),2022.0,7.0,최고의 날이었습니다.”,결혼식을 하고 1박을 롯데호텔에서 하게 되었는데 제 인생에서 최고의 하루였습니다. ...,1,2022-07-19,2022-07-19,5
1,2,스테이 인터뷰 제주,3.6,NaN,So,나홀로 여행객,"부분 오션뷰 룸 (퀸베드, 금연)",2019.0,5.0,무난함”,방음이 거의 안된다고 보시면 됩니다. 리뷰를 보고 감안하고 갔으나 예약 시 조용한 ...,2,2019-05-05,2019-05-05,1
2,1,유원호텔_서초,5.6,NaN,Sang,출장 여행객,럭셔리 스위트,2021.0,11.0,서비스는 좋았으나 객실 노후 상태가 심함”,환불 조치하여 주었음,1,2021-11-29,2021-11-29,2
3,3,블루스프링 부띠크 호텔,6.8,양호,GIL,커플/2인 여행객,스튜디오 더블,2021.0,10.0,위치는 좋지만 시설 위생상태x”,제목 그대로 위치만 좋은 곳,1,2022-09-06,2022-09-06,3
4,5,랜딩관 제주신화월드 호텔 앤 리조트,10.0,최고,YOUNGMI,청소년 동반 가족 여행객,디럭스 패밀리 트윈,2020.0,9.0,9박10일 행복했던 가족여행 ”,"결혼 30주년 기념으로 제주도 9박 10일 가족여행, 숙소도 쾌적하고 편안한 행복한...",9,2020-10-07,2020-10-07,5


In [ ]:
id2label

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

In [ ]:
label2id

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

In [ ]:
test.head()

,star,Hotel_name,review_rate,review_grade,reviewer_name,trip_type,room_type,trip_date_year,trip_date_month,title,review,trip_day,review_date,datetime
0,4,베니키아 호텔 제주,7.6,매우 좋음,동현,출장 여행객,패밀리 트윈,2022.0,8.0,출장”,출장용으로 싸고 무난,2,2022-09-20,2022-09-20
1,3,블루스프링 부띠크 호텔,5.2,NaN,Hyesu,그룹 여행객,슈페리어 트윈,2022.0,9.0,위치만 좋은 모텔급 호텔”,추천 안 합니다,1,2022-09-30,2022-09-30
2,5,롯데호텔서울 이그제큐티브 타워,10.0,최고,HANA,커플/2인 여행객,이그제큐티브 그랜드 디럭스 트윈룸 (르 살롱 이용 포함),2021.0,4.0,최고의 프리미엄 명품서비스”,서비스의 디테일이 살아있다고 느꼈어요 비싼데 전체적으로 만족도는 매우 높아요... ...,1,2021-04-27,2021-04-27
3,1,호텔 컬리넌 건대 2호점,10.0,최고,현진,출장 여행객,프리미엄룸,2019.0,10.0,키가 1개인 것만 빼면 다좋음”,깨끗하고 조용하고 시설도 낡은 거 없이 깔끔함. 근데 키가 1개라 함께 다니는 사람...,3,2019-11-04,2019-11-04
4,1,숨 게스트 하우스 제주공항,10.0,최고,HyunHo,나홀로 여행객,6 베드 도미토리 내 1인 예약 (남성 전용),2020.0,11.0,좋습니다”,좋습니다ㅏ.,2,2020-11-22,2020-11-22


In [ ]:
train_dataset=load_dataset('csv',data_files='/content/train.csv',split='train')
test_dataset=load_dataset('csv',data_files='/content/test.csv',split='train')
train_dataset=train_dataset.train_test_split(test_size=VALID_SPLIT)
#clear_output()
print(train_dataset['train'].shape,train_dataset['test'].shape,test_dataset.shape)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-daed3b3c7b66ad56/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-02ff2f0e9c444d97/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.
(2443, 15) (272, 15) (302, 14)


In [ ]:
display(train_dataset['train'])
print()
display(test_dataset)

Dataset({
    features: ['star', 'Hotel_name', 'review_rate', 'review_grade', 'reviewer_name', 'trip_type', 'room_type', 'trip_date_year', 'trip_date_month', 'title', 'review', 'trip_day', 'review_date', 'datetime', 'rate'],
    num_rows: 2443
})

Dataset({
    features: ['star', 'Hotel_name', 'review_rate', 'review_grade', 'reviewer_name', 'trip_type', 'room_type', 'trip_date_year', 'trip_date_month', 'title', 'review', 'trip_day', 'review_date', 'datetime'],
    num_rows: 302
})

# Tokenization

In [ ]:
def tokenize(model_path: str) -> Callable[[Dataset],BatchEncoding]:
    tokenizer = AutoTokenizer.from_pretrained(model_path, problem_type='multi_label_classification')
    #clear_output()
    return lambda examples: tokenizer(examples['review'], max_length=MAX_LEN, padding='max_length', truncation=True)

def one_hot(examples: Dataset) -> Dict[str,np.ndarray]:
    return {'labels':np.eye(len(LABELS))[label2id[examples['rate']]]}

def preprocess(data: Dataset, model_path: str, labeled=True) -> Dataset:
    encoded = data.map(tokenize(model_path), batched=True, remove_columns=['review'], load_from_cache_file=False)
    encoded = encoded.map(one_hot, remove_columns=['rate'], load_from_cache_file=False) if labeled else encoded
    encoded.set_format('torch')
    return encoded

# Load Model

In [ ]:
def model(model_path: str) -> AutoModelForSequenceClassification:
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        problem_type='multi_label_classification',
        ignore_mismatched_sizes=True,
        num_labels=len(LABELS),
        id2label=id2label,
        label2id=label2id,).to(device)
    #clear_output()

    return model

In [ ]:
def compute_metrics(p:EvalPrediction)->Dict[str,float]:
  predictions,labels=p
  y_pred=np.argmax(predictions,axis=1)
  y_true=np.argmax(labels,axis=1)
  return{
      'accuracy':accuracy_score(y_true=y_true,y_pred=y_pred),
  }

# Load Trainer

In [ ]:
from pathlib import Path

def make_dirs(name:str)->Dict[str,str]:
  output_dir = os.path.join('./saved/models', name)
  logging_dir = os.path.join('./saved/logger', name)
  exist_ok = name == ''
  Path(output_dir).mkdir(parents=True, exist_ok=exist_ok)
  Path(logging_dir).mkdir(parents=True, exist_ok=exist_ok)
  return output_dir, logging_dir

def training_args(model_name: str) -> TrainingArguments:
    output_dir, logging_dir = make_dirs(model_name)
    return TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        weight_decay=WEIGHT_DECAY,
        logging_dir=logging_dir,
        load_best_model_at_end=True,
        metric_for_best_model=METRIC,
    )

def trainer(dataset: Dataset, model_name: str, model_path: str, labeled=True) -> Trainer:
    data_loader = preprocess(dataset, model_path, labeled)

    return Trainer(
        model=model(model_path),
        args=training_args(model_name),
        train_dataset=data_loader['train'],
        eval_dataset=data_loader['test'],
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(3)],
    )

In [ ]:
koelectra_trainer=trainer(train_dataset,MODEL_NAME,MODEL_PATH)

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2443 [00:00<?, ?ex/s]

  0%|          | 0/272 [00:00<?, ?ex/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at jaehyeong/koelectra-base-v3-generalized-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
roberta_trainer=trainer(train_dataset,MODEL_NAME,MODEL_PATH)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/tokenizer_config.json


...

  0%|          | 0/272 [00:00<?, ?ex/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": 1,
    "1": 2,
    "2": 3,
    "3": 4,
    "4": 5
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "1": 0,
    "2": 1,
    "3": 2,
    "4": 3,
    "5": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "tokenize

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/pytorch_model.bin
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSe

# Train

In [ ]:
koelectra_trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type. If trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2443
  Num Epochs = 8
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 312


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.402130,0.430147
2,No log,0.402130,0.430147
3,No log,0.402130,0.430147
4,No log,0.402130,0.430147


The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type. If trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 272
  Batch size = 64
Saving model checkpoint to ./saved/models/koelectra/checkpoint-39
Configuration saved in ./saved/models/koelectra/checkpoint-39/config.json
Model weights saved in ./saved/models/koelectra/checkpoint-39/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored:

TrainOutput(global_step=156, training_loss=0.36604812817695814, metrics={'train_runtime': 222.2233, 'train_samples_per_second': 87.948, 'train_steps_per_second': 1.404, 'total_flos': 642797622039552.0, 'train_loss': 0.36604812817695814, 'epoch': 4.0})

In [ ]:
#reberta train
roberta_trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type. If trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2443
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 616


RuntimeError: ignored

In [ ]:
koelectra_trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type. If trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 272
  Batch size = 64


{'eval_loss': 0.40212953090667725,
 'eval_accuracy': 0.43014705882352944,
 'eval_runtime': 2.0096,
 'eval_samples_per_second': 135.35,
 'eval_steps_per_second': 2.488,
 'epoch': 4.0}

In [ ]:
test_dataset = preprocess(test_dataset, MODEL_PATH, labeled=False)
result = koelectra_trainer.predict(test_dataset)
preds = np.vectorize(id2label.get)(np.argmax(result.predictions, axis=1))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--jaehyeong--koelectra-base-v3-generalized-sentiment-analysis/snapshots/f77aac8b7a2a88515e168082dd298d9982e73edb/config.json
Model config ElectraConfig {
  "_name_or_path": "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "summary_activation": "gelu",
  "sum

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type. If trip_date_year, title, star, review_grade, reviewer_name, room_type, trip_day, review_rate, datetime, Hotel_name, trip_date_month, review_date, trip_type are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 302
  Batch size = 64


In [ ]:
len(preds)

302

In [ ]:
preds

array([3, 3, 4, 4, 5, 5, 4, 4, 5, 3, 4, 5, 3, 3, 4, 4, 5, 3, 3, 3, 3, 5,
       4, 4, 5, 5, 4, 3, 5, 2, 5, 3, 5, 5, 4, 5, 5, 4, 4, 5, 4, 4, 5, 4,
       4, 4, 5, 5, 4, 4, 4, 4, 5, 5, 5, 4, 5, 3, 5, 3, 5, 3, 4, 4, 4, 5,
       3, 3, 5, 4, 2, 5, 3, 5, 4, 3, 5, 3, 5, 3, 4, 5, 3, 4, 4, 4, 4, 2,
       4, 4, 5, 4, 5, 5, 5, 4, 5, 2, 4, 2, 4, 4, 3, 3, 3, 3, 3, 5, 4, 5,
       4, 2, 3, 5, 4, 3, 5, 4, 3, 5, 3, 5, 3, 3, 4, 5, 4, 2, 5, 3, 3, 5,
       5, 3, 4, 4, 4, 4, 3, 5, 4, 5, 4, 4, 4, 5, 3, 5, 3, 3, 3, 2, 2, 4,
       5, 3, 3, 3, 5, 4, 4, 5, 3, 4, 3, 3, 4, 2, 4, 5, 3, 4, 4, 4, 5, 2,
       3, 3, 4, 4, 5, 5, 3, 3, 5, 3, 4, 4, 4, 3, 4, 3, 4, 4, 4, 3, 5, 4,
       3, 4, 4, 5, 5, 3, 3, 3, 5, 5, 5, 5, 5, 3, 3, 4, 4, 5, 3, 3, 5, 2,
       2, 3, 3, 5, 4, 4, 3, 4, 2, 4, 5, 2, 4, 4, 4, 3, 3, 5, 5, 4, 3, 5,
       3, 3, 5, 4, 5, 3, 3, 3, 5, 3, 5, 3, 4, 5, 2, 4, 3, 2, 5, 5, 3, 4,
       3, 5, 3, 5, 4, 3, 4, 4, 4, 5, 5, 2, 5, 5, 3, 4, 3, 4, 4, 3, 2, 5,
       4, 3, 4, 5, 4, 3, 3, 3, 3, 4, 3, 5, 4, 3, 5,

In [ ]:
test_rate['pred_rate']=preds

In [ ]:
test_rate

,rate,pred_rate
0,3,3
1,2,3
2,5,4
3,5,4
4,5,5
...,...,...
297,5,5
298,5,4
299,2,3
300,1,5


In [ ]:
test_rate.to_csv('koelectra_preds.csv',index=False)